<a href="https://colab.research.google.com/github/Tara2712/Informatika/blob/main/Procesiranje_podatkov/dokon%C4%8Dna_predpriprava_podatkov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [347]:
import pandas as pd

In [348]:
df = pd.read_csv('opis_cleaned.csv', index_col='SR')

In [349]:
df.shape

(28353, 6)

In [350]:
# df.head()

In [351]:
df.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,900
OPIS,2499
DOLGI_OPIS_X,10210


In [353]:
stolpci_za_preverit = ["OPIS", "DOLGI_OPIS_X"]

num_both_null = df[(df['OPIS'].isna()) & (df['DOLGI_OPIS_X'].isna())].shape[0] #& (df['WORKLOGID'].isna())
print(f"Number of rows where both 'OPIS' and 'DOLGI_OPIS_X' are null: {num_both_null}")

Number of rows where both 'OPIS' and 'DOLGI_OPIS_X' are null: 1297


In [354]:
df_pociscen = df.dropna(subset=stolpci_za_preverit, how="all")

In [355]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,8913


In [356]:
def drop_empty_resitev(df):
    pogoj = ((df["OPIS"] == "Rešitev") | (df["OPIS"] == "Potrjujem rešitev") | (df["OPIS"] == "Zavračam rešitev")) & (df["DOLGI_OPIS_X"].isnull() | (df["DOLGI_OPIS_X"].str.strip() == ""))
    df_cleaned = df[~pogoj].copy()  # ~ znegira pogoj
    return df_cleaned

In [357]:
df_pociscen = drop_empty_resitev(df_pociscen)

In [358]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,518


In [359]:
import re
import numpy as np
def clean_and_null(text):
    if not isinstance(text, str):
        return np.nan
    cleaned = re.sub(r'img\s+[^>]*?src="data:image[^"]+"', '', text, flags=re.IGNORECASE).strip()
    return cleaned if cleaned else np.nan

df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].apply(clean_and_null)
# df_pociscen.iloc[87:91]


def odstrani_dodane_slike_v_besedilu(text):
    if not isinstance(text, str):
        return text

    return re.sub(r'img\s+[^>]*?src="data:image[^"]*$', '', text, flags=re.IGNORECASE)

df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].apply(odstrani_dodane_slike_v_besedilu) #.fillna("")

In [360]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,589


In [383]:
# df_pociscen['OPIS'].value_counts()

In [362]:
# df_pociscen.head()

In [363]:
def drop_empty_zavracam_resitev(df):
    pogoj = ((df["OPIS"] == "Zavračam rešitev") | (df["OPIS"] == "Zapiramo zahtevek")| (df["OPIS"] == "Zahtevek zapiramo") | (df["OPIS"] == "Urejeno")) & (df["DOLGI_OPIS_X"].isnull() | (df["DOLGI_OPIS_X"].str.strip() == "") )
    df_cleaned = df[~pogoj].copy()
    return df_cleaned

In [364]:
df_pociscen = drop_empty_zavracam_resitev(df_pociscen)

In [365]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,580


In [366]:
df_pociscen.shape

(18652, 6)

In [367]:
df_pociscen["OPIS"].value_counts()

,count
OPIS,
Rešitev,10978
Komentar IN,279
Komentar INF,252
Odgovor,217
Zahtevek sprejet,172
...,...
Zapis pri fakturi v delovnem toku,1
Odgovor informatika,1
sprejeto v reševanje,1


In [368]:
# df_pociscen.to_csv('2proba.csv', index=True)

In [369]:
# df_pociscen = df_pociscen['DOLGI_OPIS_X'].dropna()

In [370]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,580


In [374]:
df_pociscen = df_pociscen.dropna(subset=['DOLGI_OPIS_X'])
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,0


In [375]:
df_pociscen.shape

(18072, 6)

In [376]:
from sklearn.impute import SimpleImputer
import numpy as np

In [377]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_pociscen["OPIS"] = imputer.fit_transform(df_pociscen[["OPIS"]]).ravel()

<ipython-input-377-1c1063f251bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pociscen["OPIS"] = imputer.fit_transform(df_pociscen[["OPIS"]]).ravel()


In [378]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [379]:
df_pociscen.shape

(18072, 6)

In [380]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [381]:
df_pociscen.to_csv("df_no_nan_img.csv", index=True)

In [382]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0
